In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error

os.environ["KMP_DUPLICATE_LIB_OK"]="True"

In [2]:
def process_FAA_hourly_data(filename):
    path = os.getcwd()
    pathfile = os.path.join(path,"data",filename)
    df_temps = pd.read_csv(pathfile, skiprows=16)
    df_temps = df_temps.iloc[:,:-1]
    df_temps = df_temps.loc[df_temps[df_temps.columns[0]] != df_temps.columns[0]]
    df_temps[df_temps.columns[1]] = df_temps[df_temps.columns[1]].apply(pd.to_numeric, downcast = "integer")
    df_temps[df_temps.columns[2:]] = df_temps[df_temps.columns[2:]].apply(pd.to_numeric, downcast = "float")
    df_temps = df_temps.set_index(pd.DatetimeIndex(df_temps[df_temps.columns[0]]))
    df_temps = df_temps.drop([df_temps.columns[0]], axis=1)
    return df_temps

In [3]:
df_kphl = process_FAA_hourly_data("faa_hourly-KPHL_20120101-20190101.csv")


In [4]:
df_kphl_useful = df_kphl.iloc[:,[1,7,8,9]]
df_kphl_temp = df_kphl.iloc[:,1]

In [5]:
#df_kphl_useful contains four different variables
start = df_kphl_useful.index[0]
end = df_kphl_useful.index[-1]
idx = pd.date_range(start, end, freq='H')

df_kphl_useful = df_kphl_useful.reindex(idx, fill_value = np.nan)


In [6]:
#df_kphl_temp contains only temperature data
start = df_kphl_temp.index[0]
end = df_kphl_temp.index[-1]
idx = pd.date_range(start, end, freq='H')

df_kphl_temp = df_kphl_temp.reindex(idx, fill_value = np.nan)


In [7]:
print("Start day four variables is {}".format(df_kphl_useful.index[0]))
print("End day four variables is {}".format(df_kphl_useful.index[-1]))
print("\n\n")
print("Start day for temperature is {}".format(df_kphl_useful.index[0]))
print("End day for temperature is {}".format(df_kphl_useful.index[-1]))

Start day four variables is 2012-01-01 00:00:00
End day four variables is 2019-01-01 23:00:00



Start day for temperature is 2012-01-01 00:00:00
End day for temperature is 2019-01-01 23:00:00


In [8]:
#Get all four variables
def create_dataset(dataset, look_back=24):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :].flatten()
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [9]:
temp = np.array(df_kphl_temp)
temp = temp.reshape(-1,1)

In [10]:
#Stacked auto encoder
from keras.layers import *
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, RepeatVector, Dense, TimeDistributed

Using TensorFlow backend.


In [11]:
# split into train and test sets (67% of them are for train data)
temp = np.array(df_kphl_temp)
temp = temp.reshape(-1,1)
temp = np.nan_to_num(temp)

useful = np.array(df_kphl_useful)
useful = np.nan_to_num(useful)


In [12]:
# split into train and test sets (67% of them are for train data)

#Normalize the data
scaler_temp = MinMaxScaler(feature_range=(0, 1))
temp = scaler_temp.fit_transform(temp)

#Normalize the temperature
scaler = MinMaxScaler(feature_range=(0, 1))
useful = scaler.fit_transform(useful)


In [27]:
number_of_splits = 5
splits = TimeSeriesSplit(n_splits = number_of_splits)

In [61]:
def buildLSTMmodel(trainX, trainY, testX, testY,status):
    #build a model
    model = Sequential()
    model.add(LSTM(96, input_shape=(trainX.shape[1], trainX.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    model.fit(trainX, trainY, epochs=60, batch_size=72, validation_data=(testX, testY), verbose=2, shuffle=False)
    
    #make predict
    testPredict = model.predict(testX)
    testPredict = scaler_temp.inverse_transform(testPredict)
    if status == "Temp":
        testY = scaler_temp.inverse_transform(testY)
    else:
        testY = scaler_temp.inverse_transform([testY])
    #calculate root mean squared error
    
    if status == "Temp":
        testScore = mean_absolute_error(testY, testPredict)
    else:
        testScore = mean_absolute_error(testY[0], testPredict[:,0])
    return testScore
    

In [28]:
score = 0

for train_index, test_index in splits.split(useful):
    train = useful[train_index]
    test = useful[test_index]
    print('Observations: %d' % (len(train) + len(test)))
    print('Training Observations: %d' % (len(train)))
    print('Testing Observations: %d' % (len(test)))
    look_back = 24
    trainX, trainY = create_dataset(train,look_back)
    testX, testY = create_dataset(test, look_back)
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    s = buildLSTMmodel(trainX,trainY,testX,testY)
    score += s

print('Test Score : %.2f MSE' % (score/number_of_splits))

    
    

Observations: 20464
Training Observations: 10232
Testing Observations: 10232
Train on 10207 samples, validate on 10207 samples
Epoch 1/60
 - 6s - loss: 0.0634 - val_loss: 0.0887
Epoch 2/60
 - 2s - loss: 0.0473 - val_loss: 0.0749
Epoch 3/60
 - 2s - loss: 0.0439 - val_loss: 0.0682
Epoch 4/60
 - 2s - loss: 0.0419 - val_loss: 0.0640
Epoch 5/60
 - 2s - loss: 0.0404 - val_loss: 0.0620
Epoch 6/60
 - 2s - loss: 0.0388 - val_loss: 0.0574
Epoch 7/60
 - 2s - loss: 0.0365 - val_loss: 0.0461
Epoch 8/60
 - 2s - loss: 0.0364 - val_loss: 0.0437
Epoch 9/60
 - 2s - loss: 0.0358 - val_loss: 0.0398
Epoch 10/60
 - 2s - loss: 0.0348 - val_loss: 0.0366
Epoch 11/60
 - 2s - loss: 0.0348 - val_loss: 0.0394
Epoch 12/60
 - 2s - loss: 0.0339 - val_loss: 0.0335
Epoch 13/60
 - 2s - loss: 0.0338 - val_loss: 0.0365
Epoch 14/60
 - 2s - loss: 0.0329 - val_loss: 0.0355
Epoch 15/60
 - 2s - loss: 0.0325 - val_loss: 0.0304
Epoch 16/60
 - 2s - loss: 0.0314 - val_loss: 0.0275
Epoch 17/60
 - 2s - loss: 0.0312 - val_loss: 0.026

Epoch 15/60
 - 5s - loss: 0.0220 - val_loss: 0.0178
Epoch 16/60
 - 5s - loss: 0.0225 - val_loss: 0.0176
Epoch 17/60
 - 5s - loss: 0.0223 - val_loss: 0.0189
Epoch 18/60
 - 5s - loss: 0.0202 - val_loss: 0.0165
Epoch 19/60
 - 6s - loss: 0.0200 - val_loss: 0.0183
Epoch 20/60
 - 6s - loss: 0.0204 - val_loss: 0.0166
Epoch 21/60
 - 5s - loss: 0.0198 - val_loss: 0.0175
Epoch 22/60
 - 5s - loss: 0.0214 - val_loss: 0.0160
Epoch 23/60
 - 5s - loss: 0.0201 - val_loss: 0.0158
Epoch 24/60
 - 6s - loss: 0.0214 - val_loss: 0.0156
Epoch 25/60
 - 6s - loss: 0.0194 - val_loss: 0.0169
Epoch 26/60
 - 6s - loss: 0.0194 - val_loss: 0.0156
Epoch 27/60
 - 6s - loss: 0.0192 - val_loss: 0.0156
Epoch 28/60
 - 6s - loss: 0.0187 - val_loss: 0.0183
Epoch 29/60
 - 6s - loss: 0.0201 - val_loss: 0.0144
Epoch 30/60
 - 5s - loss: 0.0187 - val_loss: 0.0161
Epoch 31/60
 - 6s - loss: 0.0185 - val_loss: 0.0173
Epoch 32/60
 - 6s - loss: 0.0201 - val_loss: 0.0158
Epoch 33/60
 - 5s - loss: 0.0181 - val_loss: 0.0160
Epoch 34/60


Epoch 31/60
 - 10s - loss: 0.0169 - val_loss: 0.0247
Epoch 32/60
 - 10s - loss: 0.0168 - val_loss: 0.0257
Epoch 33/60
 - 10s - loss: 0.0164 - val_loss: 0.0198
Epoch 34/60
 - 10s - loss: 0.0169 - val_loss: 0.0218
Epoch 35/60
 - 11s - loss: 0.0163 - val_loss: 0.0201
Epoch 36/60
 - 10s - loss: 0.0165 - val_loss: 0.0175
Epoch 37/60
 - 11s - loss: 0.0161 - val_loss: 0.0234
Epoch 38/60
 - 10s - loss: 0.0162 - val_loss: 0.0218
Epoch 39/60
 - 9s - loss: 0.0162 - val_loss: 0.0203
Epoch 40/60
 - 10s - loss: 0.0162 - val_loss: 0.0176
Epoch 41/60
 - 11s - loss: 0.0158 - val_loss: 0.0180
Epoch 42/60
 - 10s - loss: 0.0160 - val_loss: 0.0199
Epoch 43/60
 - 9s - loss: 0.0160 - val_loss: 0.0184
Epoch 44/60
 - 9s - loss: 0.0158 - val_loss: 0.0164
Epoch 45/60
 - 9s - loss: 0.0157 - val_loss: 0.0186
Epoch 46/60
 - 9s - loss: 0.0153 - val_loss: 0.0182
Epoch 47/60
 - 10s - loss: 0.0156 - val_loss: 0.0195
Epoch 48/60
 - 9s - loss: 0.0154 - val_loss: 0.0185
Epoch 49/60
 - 9s - loss: 0.0150 - val_loss: 0.0181


In [40]:
def create_datasettemp(dataset, look_back=24):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)].flatten()
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

In [60]:
score = 0

for train_index, test_index in splits.split(useful):
    train = temp[train_index]
    test = temp[test_index]
    print('Observations: %d' % (len(train) + len(test)))
    print('Training Observations: %d' % (len(train)))
    print('Testing Observations: %d' % (len(test)))
    
    trainX, trainY = create_datasettemp(train)
    testX, testY = create_datasettemp(test)
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    s = buildLSTMmodel(trainX,trainY,testX,testY,"Temp")
    score += s

print('Test Score : %.2f MSE' % (score/number_of_splits))

    
    

Observations: 20464
Training Observations: 10232
Testing Observations: 10232
Train on 10207 samples, validate on 10207 samples
Epoch 1/60
 - 7s - loss: 0.0658 - val_loss: 0.0470
Epoch 2/60
 - 2s - loss: 0.0407 - val_loss: 0.0466
Epoch 3/60
 - 2s - loss: 0.0361 - val_loss: 0.0377
Epoch 4/60
 - 2s - loss: 0.0344 - val_loss: 0.0339
Epoch 5/60
 - 2s - loss: 0.0329 - val_loss: 0.0288
Epoch 6/60
 - 2s - loss: 0.0314 - val_loss: 0.0283
Epoch 7/60
 - 2s - loss: 0.0306 - val_loss: 0.0276
Epoch 8/60
 - 2s - loss: 0.0300 - val_loss: 0.0273
Epoch 9/60
 - 2s - loss: 0.0288 - val_loss: 0.0293
Epoch 10/60
 - 2s - loss: 0.0299 - val_loss: 0.0291
Epoch 11/60
 - 2s - loss: 0.0282 - val_loss: 0.0279
Epoch 12/60
 - 2s - loss: 0.0286 - val_loss: 0.0281
Epoch 13/60
 - 2s - loss: 0.0276 - val_loss: 0.0265
Epoch 14/60
 - 2s - loss: 0.0276 - val_loss: 0.0283
Epoch 15/60
 - 2s - loss: 0.0267 - val_loss: 0.0292
Epoch 16/60
 - 2s - loss: 0.0271 - val_loss: 0.0276
Epoch 17/60
 - 2s - loss: 0.0263 - val_loss: 0.029

Epoch 24/60
 - 4s - loss: 0.0176 - val_loss: 0.0154
Epoch 25/60
 - 5s - loss: 0.0182 - val_loss: 0.0153
Epoch 26/60
 - 5s - loss: 0.0182 - val_loss: 0.0162
Epoch 27/60
 - 4s - loss: 0.0178 - val_loss: 0.0163
Epoch 28/60
 - 5s - loss: 0.0178 - val_loss: 0.0147
Epoch 29/60
 - 5s - loss: 0.0176 - val_loss: 0.0156
Epoch 30/60
 - 5s - loss: 0.0174 - val_loss: 0.0155
Epoch 31/60
 - 4s - loss: 0.0168 - val_loss: 0.0143
Epoch 32/60
 - 4s - loss: 0.0178 - val_loss: 0.0155
Epoch 33/60
 - 4s - loss: 0.0168 - val_loss: 0.0144
Epoch 34/60
 - 4s - loss: 0.0180 - val_loss: 0.0152
Epoch 35/60
 - 4s - loss: 0.0175 - val_loss: 0.0144
Epoch 36/60
 - 5s - loss: 0.0172 - val_loss: 0.0137
Epoch 37/60
 - 5s - loss: 0.0175 - val_loss: 0.0172
Epoch 38/60
 - 5s - loss: 0.0172 - val_loss: 0.0142
Epoch 39/60
 - 5s - loss: 0.0173 - val_loss: 0.0138
Epoch 40/60
 - 4s - loss: 0.0172 - val_loss: 0.0141
Epoch 41/60
 - 5s - loss: 0.0171 - val_loss: 0.0149
Epoch 42/60
 - 5s - loss: 0.0171 - val_loss: 0.0140
Epoch 43/60


Epoch 49/60
 - 10s - loss: 0.0155 - val_loss: 0.0147
Epoch 50/60
 - 10s - loss: 0.0155 - val_loss: 0.0151
Epoch 51/60
 - 9s - loss: 0.0154 - val_loss: 0.0149
Epoch 52/60
 - 10s - loss: 0.0153 - val_loss: 0.0143
Epoch 53/60
 - 11s - loss: 0.0154 - val_loss: 0.0143
Epoch 54/60
 - 11s - loss: 0.0153 - val_loss: 0.0143
Epoch 55/60
 - 8s - loss: 0.0153 - val_loss: 0.0145
Epoch 56/60
 - 9s - loss: 0.0153 - val_loss: 0.0141
Epoch 57/60
 - 8s - loss: 0.0153 - val_loss: 0.0148
Epoch 58/60
 - 8s - loss: 0.0153 - val_loss: 0.0149
Epoch 59/60
 - 9s - loss: 0.0152 - val_loss: 0.0148
Epoch 60/60
 - 9s - loss: 0.0152 - val_loss: 0.0144
[73.]
[73.595695 75.06     76.60495  75.70578  73.82044  72.43927  70.10324
 68.012566 65.893394 64.07839  63.381977 62.610146 61.818604 60.70573
 58.70461  57.783966 57.230778 58.629997 59.2184   59.774254]
Test Score : 1.84 MSE
